In [1]:
import pandas as pd

In [2]:
SET_ID = "v3_fdf95_added"

In [3]:
df = pd.read_csv("/kuacc/users/mugekural/workfolder/dev/git/cogeval/results/SNLI-lalor/"+SET_ID+"_truefalse.csv")

In [4]:
questions = dict()
sample_ids = dict()

for ind, row in df.iterrows():

    sample_ids[ind] = row['sample_id']

    sentence1 = row['sentence_1']
    sentence2 = row['sentence_2']

    text = f"""<b>Premise:&nbsp;</b>{sentence1}<br>
<br>
<b>Hypothesis:&nbsp;</b>{sentence2}
"""

    questions[row['sample_id']] = text

    print(row['sample_id'],'\n',text)

0 
 <b>Premise:&nbsp;</b>A little boy is opening gifts surrounded by a group of children and adults.<br>
<br>
<b>Hypothesis:&nbsp;</b>The boy is being punished

1 
 <b>Premise:&nbsp;</b>People playing cricket in the park, pine trees in the back.<br>
<br>
<b>Hypothesis:&nbsp;</b>People are playing sports in the park, near the pine trees

3 
 <b>Premise:&nbsp;</b>A group of dancers are performing.<br>
<br>
<b>Hypothesis:&nbsp;</b>The audience is silent.

5 
 <b>Premise:&nbsp;</b>A little girl in pink stands in a yellow hallway.<br>
<br>
<b>Hypothesis:&nbsp;</b>The girl is standing outside her apartment.

6 
 <b>Premise:&nbsp;</b>A young boy in a sweatshirt is doodling on a piece of paper.<br>
<br>
<b>Hypothesis:&nbsp;</b>The class pictures are on display.

8 
 <b>Premise:&nbsp;</b>a toddler playing with a toy car next to a dog.<br>
<br>
<b>Hypothesis:&nbsp;</b>A toddler plays with toy cars while his dog sleeps.

9 
 <b>Premise:&nbsp;</b>A man in and a woman are holding their towels at th

In [5]:
models = ['random_noisy', 'tfidf', 'lstm', 'roberta_calibrated', 'davinci', 'human']

In [6]:
model_probs = dict()
for model in models:
  model_df = pd.read_csv("/kuacc/users/mugekural/workfolder/dev/git/cogeval/data/machine/SNLI-lalor/"+SET_ID+"/" + model +".csv")
  model_probs[model] = model_df[['c', 'n', 'e']].copy()
num_samples = model_probs['human'].shape[0]

In [7]:
def get_label(row):

  if row['c'] < 0.01:  
    c_val = "0.01> contradiction"
  else:
    c_val = str(round(row['c'],2)) + " contradiction"

  if row['c'] >= 0.995:
    c_val="0.99< contradiction"

  if row['n'] < 0.01:
    n_val = "0.01> neutral"
  else:
    n_val = str(round(row['n'],2)) + " neutral"

  if row['n'] >= 0.995:
    n_val = "0.99< neutral"

  if row['e'] < 0.01:
    e_val = "0.01> entailment"
  else:
    e_val = str(round(row['e'],2)) + " entailment"   

  if row['e'] >= 0.995:
    e_val = "0.99< entailment"

  return c_val + " | " + n_val + " | " + e_val

for model in models:
  model_probs[model]['label_str'] = model_probs[model].apply(get_label, axis=1)

model_probs['davinci']

,c,n,e,label_str
0,0.936637,0.063113,0.000250,0.94 contradiction | 0.06 neutral | 0.01> enta...
1,0.000243,0.265058,0.734699,0.01> contradiction | 0.27 neutral | 0.73 enta...
2,0.000032,0.002295,0.997673,0.01> contradiction | 0.01> neutral | 0.99< en...
3,0.000056,0.999938,0.000006,0.01> contradiction | 0.99< neutral | 0.01> en...
4,0.000013,0.999974,0.000006,0.01> contradiction | 0.99< neutral | 0.01> en...
...,...,...,...,...
85,0.985226,0.014733,0.000041,0.99 contradiction | 0.01 neutral | 0.01> enta...
86,0.000299,0.999658,0.000043,0.01> contradiction | 0.99< neutral | 0.01> en...
87,0.000011,0.999977,0.000006,0.01> contradiction | 0.99< neutral | 0.01> en...
88,0.003722,0.996128,0.000150,0.01> contradiction | 0.99< neutral | 0.01> en...


In [8]:
survey_header = """[[AdvancedFormat]]

[[Block]]\n
"""


In [9]:
import random
#modelIDS = {'random_noisy':0, 'tfidf':1,'lstm':2,'roberta_calibrated':3,'davinci':4,'human':5}


In [10]:
with open("survey_userstudy_"+SET_ID+".txt", "w") as f:
  f.write(survey_header)

with open("survey_userstudy_"+SET_ID+".txt", "a") as f:

  for ind, row in df.iterrows():

    question_text = f"""[[Question:MC]]
[[ID:q{ind+1}]]
{questions[row['sample_id']]}\n"""

    f.write(question_text)
    f.write("[[AdvancedChoices]]\n")
    #random.shuffle(models)
    for mid,model in enumerate(models):
      #mid = modelIDS[model]
      #if len(row['true_machs']) == 2:
      #    f.write("[[Choice:0]]\n")   
      #elif (mid in  [int(mid) for mid in row['true_machs'][1:-1].split(',')]):
      #    f.write("[[Choice:1]]\n")  
      #else:
      #    f.write("[[Choice:0]]\n") 
      f.write("[[Choice]]\n") 


      f.write(f"{model_probs[model]['label_str'].tolist()[row['sample_id']]}<br>")
      img_url = f"https://storage.googleapis.com/cogeval-bucket1/m{mid}/{row['sample_id']}.png\""
      f.write(f"<img src=\"{img_url}/>\n")
